Lambda School Data Science

Jeremy Glasser - Build Week Project Unit 2
Explore and experiment with select market data and linear regression.
Looking for features that correlate to large changes in absolute value by date.
End goal: check related time-series data from other sources given the highest change dates. 
Looking for what was seen that caused the correction, or crash in the cases of the obvious Black Swan type events.
---

In [1]:
import sys
!pip install -q --user category_encoders==2.*
!pip install -q --user pandas-profiling==2.*
!pip install -q --user lazypredict
!pip install -q --user Quandl
!pip install -q --user pdpbox
!pip install -q --user shap
!pip install -q --user matplotlib
!pip install -q --user talib

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\17708\\AppData\\Roaming\\Python\\Python37\\site-packages\\matplotlib\\_image.cp37-win_amd64.pyd'
Check the permissions.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\17708\\AppData\\Roaming\\Python\\Python37\\site-packages\\matplotlib\\_image.cp37-win_amd64.pyd'
Check the permissions.

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\17708\\AppData\\Roaming\\Python\\Python37\\site-packages\\matplotlib\\_image.cp37-win_amd64.pyd'
Check the permissions.

ERROR: Could not find a version that satisfies the requirement talib (from versions: none)
ERROR: No matching distribution found for talib


In [2]:
# Importing main packages:

import sklearn
import numpy as np
import pandas as pd
import matplotlib as mpl

# Importing Viz:

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from ipywidgets import interactive, IntSlider, FloatSlider

# Importing Model Types:

from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.linear_model import OrthogonalMatchingPursuitCV

#Importing Model Split/Subsets for Training/Testing:

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV # Hyperparameter tuning

# Importing Pipeline components:

from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from category_encoders import OrdinalEncoder
from category_encoders import OneHotEncoder

# Importing Metrics for Model explanability

from sklearn import metrics
from math import floor, ceil

from sklearn.inspection import permutation_importance
from sklearn.model_selection import cross_val_score, validation_curve
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import shap

from talib import add_all_ta_features

# Data Import from Fed, and custom financial instruments/assets dataset from my Github:
import quandl

DATA_PATH = 'https://raw.githubusercontent.com/Artifice-Shell/U2BlackSwan/main/'

#mydata = quandl.get("FRED/GDP")
#quandl.ApiConfig.api_key = "tNMeunHf8-3SD87apRvF"

#quandl.bulkdownload("DatasetName")

#Retrieve entire Table: (Returns max 1M rows)
#quandl.get_table('MER/F1', paginate=True)

#Export to Zip to get more data rows:
#quandl.export_table('MER/F1')
#Put in in a specific place:
#quandl.export_table('MER/F1', filename='/my/path/db.zip')



ImportError: cannot import name '_api' from 'matplotlib' (C:\Users\17708\AppData\Roaming\Python\Python37\site-packages\matplotlib\__init__.py)

In [ ]:
# Quandl - Report/Name
#REPORT	NAME
#H.3	Aggregate Reserves of Depository Institutions and the Monetary Base
#E.15	Agricultural Finance Databook
#H.8	Assets and Liabilities of Commercial Banks in the U.S.
#Charge-Off and Delinquency Rates on Loans and Leases at Commercial Banks	
#Commercial Paper	
#G.19	Consumer Credit
#Corporate Medium-Term Notes	
#E.16	Country Exposure Lending Survey
#H.4.1	Factors Affecting Reserve Balances
#Z.1	Financial Accounts of the United States
#G.20	Finance Companies
#H.10	Foreign Exchange Rates
#Household Debt Service and Financial Obligations Ratios	
#G.17	Industrial Production and Capacity Utilization
#International Summary Statistics	
#H.6	Money Stock Measures
#Mortgage Debt Outstanding	
#H.15	Selected Interest Rates
#Survey of Consumer Finances	
#E.2	Terms of Business Lending

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [x] Choose your target. Which column in your tabular dataset will you predict?
      Starting with 'change_abs' - looking to identify what is most strongly correlated to the largest values there.
- [x] Is your problem regression or classification?
      It is a regression problem.
- [ ] How is your target distributed?
    - Classification: How many classes? Are the classes imbalanced?
    - Regression: Is the target right-skewed?
- [ ] Choose your evaluation metric(s).
    - Classification: Is your majority class frequency >= 50% and < 70% ? If so, you can just use accuracy if you want. Outside that range, accuracy could be misleading. What evaluation metric will you choose, in addition to or instead of accuracy?
    - Regression: Will you use mean absolute error, root mean squared error, R^2, or other regression metrics?
- [ ] Choose which observations you will use to train, validate, and test your model.
    - Are some observations outliers? Will you exclude them?
    - Will you do a random split or a time-based split?
- [ ] Begin to clean and explore your data.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" future information?

If you haven't found a dataset yet, do that today. [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2) and choose your dataset.

Some students worry, ***what if my model isn't “good”?*** Then, [produce a detailed tribute to your wrongness. That is science!](https://twitter.com/nathanwpyle/status/1176860147223867393)

# Generating the Hypothesis: 

***'Or how I learned to love possibly 'Boiling the Ocean' to search for the needle in the Black Swan stack, but not yet...'***

## What I'm looking at, and why I'm looking at it:

I am looking at the time-series data extending back as far as it existed, for multiple asset types, and Indexes related to the US Financial Markets and Currency Values. 

The assets I chose initially were based on a basket of things I have noted over my lifetime have a tendency to react in a volatile way to certain economic triggers or flashpoints. A needle that pierced all their hearts? A common thread? Who is the hunter? Let's see what we can see.

A good PDF sample of what a project/outcome should look like - and more information on the type of quantitative/prescriptive modeling I want to build, and will be doing (when I'm this good at it): https://papers.ssrn.com/sol3/papers.cfm?abstract_id=3538891

In [ ]:
#Not wrangling at present - only running a couple new libraries to review the data.

pd.set_option('use_inf_as_na', True)

df = pd.read_csv(DATA_PATH+'master.csv')
dfSMA = pd.read_csv(DATA_PATH+'master.csv')
df['asset'] = df['asset'].astype("category")
df["class"] = df["class"].astype("category")

dfSMA['asset'] = dfSMA['asset'].astype("category")
dfSMA["class"] = dfSMA["class"].astype("category")

cols = ['open', 'high', 'low', 'close', 'adj_close', 'vol']

# Setting a multi-index on the date and the asset (This will be important in later versions.)
df = df.set_index(['date','asset'])

# Setting a single index on the date for the SMA and other metrics.
#dfSMA = dfSMA.set_index(['date'])

# pass them to df.replace(), specifying each char and it's replacement:
df[cols] = df[cols].replace({'\$': '', ',': ''}, regex=True)
dfSMA[cols] = dfSMA[cols].replace({'\$': '', ',': ''}, regex=True)

# Printing the head for the main df:
df.head()

In [ ]:
# No difference here yet:
dfSMA.head()

In [ ]:
dfSMA.info()

In [ ]:
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
dfSMA[cols] = dfSMA[cols].apply(pd.to_numeric, errors='coerce')

In [ ]:
df.info()

In [ ]:
# Here I'm doing a little bit of the initial feature engineering for the datasets. 
# Metrics like: 
#  Change in value from open to close
#  Quantity of range between high and low
#  The Trading Volume represented as a logistic value.

df['spread'] = df['high'] - df['low']
df['change'] = df['open'] - df['close']
df['change_abs'] = df['change'].abs()
df = df.dropna()

df['vol_log'] = df.vol.apply(np.log)
df['prior_vol_chng'] = df.groupby('asset').vol.diff()
df['20_day_change'] = df.groupby('asset').vol.diff(20)

df.iloc[51:72]

In [ ]:
dfSMA = dfSMA.dropna()

dfSMA['spread'] = dfSMA['high'] - dfSMA['low']
dfSMA['change'] = dfSMA['open'] - dfSMA['close']
dfSMA['change_abs'] = dfSMA['change'].abs()

# Using the alternate single index dataframe for SMA, I can now see the SMA_20, and SMA_50.

dfSMA['SMA_20'] = talib.SMA(dfSMA['close'], timeperiod=20)
dfSMA['SMA_50'] = talib.SMA(dfSMA['close'], timeperiod=50)

# Adding in the Exponential Moving Average as well:

dfSMA['EMA_20'] = talib.EMA(dfSMA['close'], timeperiod=20)
dfSMA['EMA_50'] = talib.EMA(dfSMA['close'], timeperiod=50)

# Adding in the log of the volume here also:
dfSMA['vol_log'] = dfSMA.vol.apply(np.log)
dfSMA.iloc[51:72]

In [ ]:
# A categorical scatterplot based on the asset's Class: Here using Volume on the vertical, with change_abs on the horizontal.

fig = px.scatter(dfSMA, x = 'SMA_20', y = 'change_abs', color = 'class', trendline="ols")
fig.update_xaxes(range=[0, 15000])
fig.update_yaxes(range=[0, 500])
fig.show()

results = px.get_trendline_results(fig)
print(results)

In [ ]:
# A categorical scatterplot based on the asset's Class: Here using Volume on the vertical, with change_abs on the horizontal.

fig = px.scatter(dfSMA, x = 'SMA_50', y = 'change_abs', color = 'class', trendline="ols")
fig.update_xaxes(range=[0, 15000])
fig.update_yaxes(range=[0, 500])
fig.show()

results = px.get_trendline_results(fig)
print(results)

In [ ]:
# Really ugly and useless chart, but I made it anyway.

plt.plot(dfSMA['close'], color='blue', label='Daily Close Price')
plt.plot(dfSMA['SMA_20'], color='green', label='SMA 20')
plt.plot(dfSMA['SMA_50'], color='red', label='SMA 50')
plt.legend()
plt.title('Simple Moving Averages')
plt.show()

In [ ]:
# Looking at the dataset with a facet grid: (I made one, but I broke it.)

#sns.catplot(x="vol_log", y="change_abs", kind="boxen",data=df.sort_values("class"))

# II. Split Data

**Task 2:** Splitting dataFrame `df` into a feature matrix `X` and the target vector `y`. Predicting `'change_abs'`.

In [ ]:
target = 'change_abs'
drop_cols = ['open', 'high', 'low', 'close', 'adj_close','change','vol']

y1 = df[target]
X1 = df.drop(columns = target)

y2 = dfSMA[target]
X2 = dfSMA.drop(columns = target)
X_train.head()

In [ ]:
# Problem: -inf's in the logs - adding pd.set_option('use_inf_as_na', True) up top.

In [ ]:
X1 = X1.drop(columns = drop_cols)

X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=.3, random_state=42)

X2 = X2.drop(columns = drop_cols)

X1_train, X1_test, y1_train, y1_test = train_test_split(X2, y2, test_size=.4, random_state=42)
X1_train.head()

In [ ]:
pd.set_option('use_inf_as_na', True)


**Task 3:** Using a randomized split, divide `X` and `y` into a training set (`X_train`, `y_train`) and a validation set (`X_val`, `y_val`).

In [ ]:
X_train

In [ ]:
# Commenting out as results take considerable time, and have been documented in a cell below.

#reg = LazyRegressor(predictions=True)
#models_reg, predictions = reg.fit(X_train, X_test, y_train, y_test)

In [ ]:
#Results of the LazyRegressor, put into a field to make copy/paste work for Excel.
'''	Adjusted R-Squared	R-Squared	RMSE	Time Taken
Model				
GradientBoostingRegressor	0.80	0.80	14.57	7.71
MLPRegressor	0.79	0.79	14.81	22.61
OrthogonalMatchingPursuit	0.79	0.79	14.82	0.06
HuberRegressor	0.79	0.79	14.83	0.75
LassoCV	0.79	0.79	14.83	0.54
LinearSVR	0.79	0.79	14.84	13.08
RidgeCV	0.79	0.79	14.84	0.09
Ridge	0.79	0.79	14.84	0.07
BayesianRidge	0.79	0.79	14.84	0.09
TransformedTargetRegressor	0.79	0.79	14.84	0.07
LinearRegression	0.79	0.79	14.84	0.06
Lars	0.79	0.79	14.84	0.08
LassoLarsIC	0.79	0.79	14.84	0.08
LarsCV	0.79	0.79	14.84	0.18
LassoLarsCV	0.79	0.79	14.84	0.16
OrthogonalMatchingPursuitCV	0.79	0.79	14.84	0.14
Lasso	0.79	0.79	14.85	0.07
ElasticNetCV	0.79	0.79	14.86	0.56
SGDRegressor	0.79	0.79	14.95	0.16
RandomForestRegressor	0.79	0.79	15.08	27.52
KNeighborsRegressor	0.79	0.79	15.12	1.26
RANSACRegressor	0.78	0.78	15.32	0.19
XGBRegressor	0.78	0.78	15.33	3.45
BaggingRegressor	0.77	0.77	15.57	2.86
ExtraTreesRegressor	0.76	0.76	15.86	14.89
AdaBoostRegressor	0.75	0.75	16.29	1.88
HistGradientBoostingRegressor	0.71	0.71	17.70	0.67
ElasticNet	0.70	0.70	17.79	0.07
LGBMRegressor	0.70	0.70	17.84	0.40
ExtraTreeRegressor	0.65	0.65	19.38	0.21
DecisionTreeRegressor	0.64	0.64	19.59	0.47
GeneralizedLinearRegressor	0.61	0.61	20.38	0.21
TweedieRegressor	0.61	0.61	20.38	0.19
NuSVR	0.61	0.61	20.40	1514.61
SVR	0.61	0.61	20.40	1096.83
PoissonRegressor	0.20	0.20	29.27	0.28
PassiveAggressiveRegressor	0.15	0.15	30.12	0.14
LassoLars	-0.00	-0.00	32.65	0.06
DummyRegressor	-0.00	-0.00	32.65	0.06'''

# Making Pipeline based models to compare based on what LazyPredict recommended:

*I'm aware that this makes the plotting more complicated, and I can't seem to find regression samples where we plotted from Pipelines, I am only finding those on Classification Models.*

In [ ]:
model_MLP = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    MLPRegressor()
)

model_MLP.fit(X_train, y_train)

expected_y  = y_test
predicted_y = model_MLP.predict(X_test)

In [ ]:
model_MLP_score = model_MLP.score(X_test, y_test)
test_set_rmse = (np.sqrt(mean_squared_error(expected_y, predicted_y)))

###OPTIONAL TO print('Linear Regression R^2:', model_lr_r2_score)
print(metrics.r2_score(expected_y, predicted_y)),
print(metrics.mean_squared_error(expected_y, predicted_y))
print(model_MLP_score)

In [ ]:
model_GBR = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    GradientBoostingRegressor()
)

model_GBR.fit(X_train, y_train)

expected_y  = y_test
y_pred = model_GBR.predict(X_test)

# A few different Pipelines to compare:

Built out some of the higher scoring models from the LazyPredict Regressor to step through them and compare them from their pipelines.


In [ ]:
from sklearn.preprocessing import StandardScaler

model_lr = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(),
    StandardScaler(with_mean=True, with_std=True),
    LinearRegression()
    )

model_GBR = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(),
    StandardScaler(with_mean=True, with_std=True),
    GradientBoostingRegressor()
    )

model_RFR = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(),
    StandardScaler(with_mean=True, with_std=True),
    RandomForestRegressor()
    )

model_MLP = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(),
    StandardScaler(with_mean=True, with_std=True),
    MLPRegressor()
    )

model_RCV = make_pipeline(
    OneHotEncoder(), 
    SimpleImputer(),
    StandardScaler(with_mean=True, with_std=True),
    RidgeCV()
    )



# Step 3: Fit predictor on the (training) data
model_lr.fit(X_train, y_train)
model_GBR.fit(X_train, y_train)
model_MLP.fit(X_train, y_train)
model_RCV.fit(X_train, y_train)
model_RFR.fit(X_train, y_train)

In [ ]:

print("Linear Regression Model coefficients:\n")
for i in range(X.shape[1]):
    print(X.columns[i], "=", model_lr.named_steps["linearregression"].coef_[i].round(4))

In [ ]:
print("Ridge Cross-validation Model coefficients:\n")
for i in range(X.shape[1]):
    print(X.columns[i], "=", model_RCV.named_steps["ridgecv"].coef_[i].round(4))

In [ ]:
#%matplotlib notebook
#matplotlib.pyplot.ion()
plt.figure(figsize=(10,10))
sns.regplot(expected_y, predicted_y, fit_reg=True, scatter_kws={"s": 100})

# Baseline MAE - it's the number to beat.

Since this is a **Regression** problem we are using the Baseline MAE as the initial accuracy benchmark for the data models to beat. If they cannot, then you need to add more Features/Independent Variables, increase the sample size, address missing values/outliers, try a different model, and maybe all of the above.

**tl;dr:** *It's y_train's mean **VS** the mean of the training model's predicted y. It's the mean of the difference (absolute value, no +/- meaning) for each observation for X in X_train.*

For Regression the Baseline MAE is calculated as the first metric for evaluating a model's performance. The difference
between the trained result's predicted value y (y_pred) given each observation for X in X_train, is compared to the mean for y_train (add or subtract is irrelevant). The difference between the y_train **mean** and the predicted value is expressed as an **absolute** value, and defined as the **error**. 

We have seen how that works in a tabular dataset, and this is also conceptually easy to seen in Excel as well (which I will not demonstrate here.)



In [ ]:
y_pred = [y_train.mean()]*len(y_train)
mean_absolute_error(y_train, y_pred)
#R squared is zero, since R Squared tells you how good the model is compared to the mean baseline
print('R Squared:', r2_score(y_train, y_pred))

In [ ]:
#Defining the y_pred for Regression:

y_pred = [y_train.mean()]*len(y_train)


baseline_mae = mean_absolute_error(y_train, y_pred)

print('Mean Absolute Change:', y_train.mean())
print('Baseline MAE:', baseline_mae)

In [ ]:
model_rf = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    RandomForestRegressor()
)

model_rf.fit(X_train, y_train)

In [ ]:
model_GBR = make_pipeline(
    OrdinalEncoder(), 
    SimpleImputer(), 
    GradientBoostingRegressor()
)

model_GBR.fit(X_train, y_train)

# V. Check Metrics

**Calculate the training and testing metrics for the models to be compared:**

**Using the Baseline Mean Absolute Error (MAE) we calculated earlier, check the Training and Testing MAE:**

**Metrics and Results:**

Calculate the training and test mean absolute error for the models.


In [ ]:
# Comparison of the Training MAE to the Test MAE for the Random Forest Regressor:

training_mae_rf = mean_absolute_error(y_train, model_rf.predict(X_train)).round(3)
test_mae_rf = mean_absolute_error(y_test, model_rf.predict(X_test)).round(3)

print('Training MAE:', training_mae_rf)
print('Test MAE:', test_mae_rf)

In [ ]:
# Comparison of the Training MAE to the Test MAE for the Gradient Boosting Regressor:

training_mae = mean_absolute_error(y_train, model_GBR.predict(X_train)).round(3)
test_mae = mean_absolute_error(y_test, model_GBR.predict(X_test)).round(3)

print('Training MAE:', training_mae)
print('Test MAE:', test_mae)

# Checking Feature Importances for the Random Forest Reg

*The most important feature is the spread, which is based on the High/Low, not the Open/Close. So it's not a direct observation, but I would expect it would have a high feature importance, there may be a way to alter it by dividing the volume by it to make a ratio, but I'm not sure if that's worth doing.*

In [ ]:
# Get feature importances
get_feat_imp = model_rf.named_steps['randomforestregressor']
importances = pd.Series(get_feat_imp.feature_importances_, X_train.columns)

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh(color='grey');

In [ ]:
# Get feature importances
get_feat_imp_GBR = model_GBR.named_steps['gradientboostingregressor']
importances = pd.Series(get_feat_imp_GBR.feature_importances_, X_train.columns)

# Plot feature importances
%matplotlib inline
import matplotlib.pyplot as plt

n = 20
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh(color='grey');

# In the case of the Gradient Boosting Regressor, it doesn't seem to consider the volume as important, let's try a different viz:

In [ ]:
f, ax = plt.subplots(figsize=(11, 7))
sns.countplot(y="class", data=df, color="c")

# How the models compared using MAE:

**Random Forest Regressor:** shows a significant difference between the training and testing mean absolute error, which would mean it didn't do as well on the test data as it did on the training data. The absolute error quantitatively in terms of the dataset is not a large one, but less error in Quantitative Financial Analytics is clearly better, and if you're expecting to be off by ~$1.07, and you are nearly ~$2.7 off, depending on the strategy employed by the trading bot, you may be out of a whole lot of cash even though it's not a $500 difference.

**Gradient Boosting Regressor:** shows a much smaller different between the training and testing MAE, as close as to be no difference at all, though still a slight difference. This could be useful where the Random Forest Regressor's difference doesn't match the risk profile for the usecase.**



**Comparing R2 scores for the models:**

Calculate the training and test  R2  score for the models.

In [ ]:
# Comparison of the Training R2 to the Test MAE for the Random Forest Regressor:

training_rf_r2 = mean_squared_error(y_train, model_rf.predict(X_train), squared=False).round(3)
test_rf_r2 = mean_squared_error(y_test, model_rf.predict(X_test), squared=False).round(3)

print('Random Forest - Training Mean Squared Error / R2:', training_rf_r2)
print('Random Forest - Test Mean Squared Error / R:', test_rf_r2)

In [ ]:
# Comparison of the Training MAE to the Test MAE for the Gradient Boosting Regressor:


training_r2 = mean_squared_error(y_train, model_GBR.predict(X_train), squared=False).round(3)
test_r2 = mean_squared_error(y_test, model_GBR.predict(X_test), squared=False).round(3)

print('Gradient Boosted - Training Mean Squared Error / R', training_r2)
print('Gradient Boosted - Test Mean Squared Error / R:', test_r2)

# Comparison using the R2:

**Random Forest Regressor**: Similar theme here as with the MAE given that it's related that should be no surprise.

**Gradient Boosting Regressor**: Again the performance of the Gradient Boosting Regressor between the training and the testing was a tighter spread, however, it was a higher value - so there's a clear trade-off between the two models and the choice of which to use would come down to strategy (or insanity of using either.)

**Overall:** Since the Absolute Error is low for both, I could see where this would be useful, however there is more to this than meets the eye, and I'll be getting to that in later revisions. Chiefly, the Time-Series nature of the predictions. If the predicted absolute change is based on a prior period, and when modeled it demonstrates that it cannot be safely acted on, then there would be no value in the prediction. The ultimate goal of anything like this is to create a 'leading indicator', not a 'lagging indicator' for reasons that are clear to any investor.

# Model Performances

**Overall:** Since the Absolute Error is relatively low for both, I could see where this could be useful. However there is more to this than meets the eye, and I'll be getting to that in later revisions. 

The primary caveat is that this is ultimately only useful when looked at through the time-series lens based on the nature of the predictions and the intent of predicting anything at all.

If the predicted absolute change is based on a prior period, and when modeled it demonstrates that it cannot be safely acted on, then there would be no value in the prediction. 

The ultimate goal of anything of this sort is to create a 'leading indicator', not a 'lagging indicator' for reasons that are clear to any investor.

#MORE COMMENTARY IF NEEDED..

# Future Plans (before running away from this dataset):

** IDK... figure something out here... or don't because if it was this simple, we'd all be rich, or just the people who solved it would be. It's Chaos Theory, it's all covered by Numberphile and Veritasium, and ThreeBlue1Brown, and many more... so maybe there's a secret in here somewhere, and I just have to be crazy enough to try to find it.

**Note:** I've done pretty crazy things before, and gotten good results from it - sometimes not knowing better is all the difference there is between a lucky autodidact, and an unlucky Professor. One of them was dumb enough to try... and fail repeatedly... until they stopped failing and grew wings. Maybe not today, maybe not tomorrow, maybe not ever... but not not ever. With the right tools and new meaning to measures, what is possible changes - and I'm not above using Quantum Compute resources to throw sand in the eyes of well-heeled Quants.

In [ ]:
#Write More Code or Bust

In [ ]:
# Viz of Top Ten Coefficients for Regression:
%matplotlib inline
from category_encoders import OneHotEncoder
from category_encoders import OrdinalEncoder
from matplotlib import pyplot
model_lr_ns = model_lr.named_steps['linearregression']
#encoder_ns = model_MLP.named_steps['ordinalencoder']

encoder = OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train)
X_val_encoded = encoder.transform(X_test)
X_train_encoded.shape, y_train.shape, X_val_encoded.shape, y_test.shape

coefficients = pd.Series(model_lr_ns.coef_[0], X_train.columns)
features = X.columns
feat_imp = pd.Series(coefficients, index=features).abs()

feat_imp.tail(5).plot(kind='barh')
plt.title('Top Ten Coefficients for Linear Regression');

# Adding Shap for Model Explainability on a RandomForestRegressor.

**Now that I've fit a new model for the RandomForestRegressor I will be reviewing that with the Shap Tree Explainer with JS visualization.**

In [ ]:
'''explainer =  shap.Explainer(model_lr.named_steps["linearregression"],masker=shap.maskers.Impute(data=X_train),

                                  feature_names=X_train.columns)

shap_values = explainer.shap_values(model_lr[:-1].transform(X_train))

 

# print the JS visualization code to the notebook

shap.initjs()

# plot the SHAP values for the output of the first instance

shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:], link="logit")'''

In [ ]:
'''row = X_train.tail(1)

explainer =  shap.LinearExplainer(model_GBR, data=X_train, masker=shap.maskers.Impute(data=X_train),
                                  feature_names=X_train.columns, algorithm="linear")
shap_values = explainer(X_test, nsamples=100)


# print the JS visualization code to the notebook
shap.initjs()

shap.force_plot(
    base_value = explainer.expected_value,
    shap_values = shap_values(X_test, nsamples=100),
    features = row
)

# plot the SHAP values for the Setosa output of the first instance
shap.force_plot(explainer.expected_value[0], shap_values[0][0,:], X_test.iloc[0,:], link="logit")'''

In [ ]:
#This one is running too slowly, skipping it for now:
'''explainer =  shap.TreeExplainer(model_RFR.named_steps["randomforestregressor"],masker=shap.maskers.Impute(data=X_train),

                                  feature_names=X_train.columns, algorithm="linear")

shap_values = explainer.shap_values(model_RFR[:-1].transform(X_train))

 

# print the JS visualization code to the notebook

shap.initjs()


# plot the SHAP values for the output of the first instance

shap.force_plot(explainer.expected_value, shap_values[0,:], X.iloc[0,:], link="logit")'''

In [ ]:
print("Gradient Boosted Regression Model coefficients:\n")    
for i in range(X.shape[1]):
    print(X['class'] [i], "=", model_GBR.named_steps["gradientboostingregressor"].feature_importances_[i].round(6))

print("\n")
    
print("Random Forest Regressor Model coefficients:\n")    
for i in range(X.shape[1]):
    print(X.columns[i], "=", model_RFR.named_steps["randomforestregressor"].feature_importances_[i].round(6))

In [ ]:
print("Multilayer Perceptron (NeuralNet) Model coefficients:\n")    
for i in range(X.shape[1]):
    print(X.columns[i], "=", "\n", model_MLP.named_steps["mlpregressor"].get_params(deep=True))
    

In [ ]:
model_MLP.predict(X_test)
print("\n")
model_GBR.predict(X_test)

# Issues with ploting from a pipeline in various ways, attempting to resolve that: 

Appears to be something to do with the 'class' and 'asset' data colums, and I want it grouped by them.
The charts were working in a prior version, not sure what went wrong, but it's 'Str' related.

In [ ]:
###  As seen here, there is more than one way to achieve the same result, 
## in this case I used the i.loc within a forloop/list comprehension, and the 'Stock'...
#  entry in the class column was forced to be looked at, where it needed to group by them.

models_reg["R-Squared"] = [0 if i < 0 else i for i in models_reg.iloc[:,0] ]
                            
plt.figure(figsize=(10,10))
sns.set_theme(style="whitegrid")
ax = sns.barplot(x=models_reg.index, y="R-Squared", data=models_reg)
ax.set(ylim=(0,1))
plt.xticks(rotation=90)

In [ ]:
encoder = OneHotEncoder(use_cat_names=True)
X_train_encoded = encoder.fit_transform(X_train['asset'])
X_test_encoded = encoder.transform(X_test['asset'])
X_train_encoded.shape, y_train.shape, X_test_encoded.shape, y_test.shape

model_GBReg = ensemble.GradientBoostingRegressor(**params)
model_GBReg.fit(X_train_encoded, y_train)

mse = mean_squared_error(y_test, model_GBReg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

test_score = np.zeros((params['n_estimators'],), dtype=np.float64)
for i, y_pred in enumerate(model_GBReg.staged_predict(X_test)):
    test_score[i] = model_GBReg.loss_(y_test, y_pred)

fig = plt.figure(figsize=(6, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, model_GBReg.train_score_, 'b-',
         label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
         label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')
fig.tight_layout()
plt.show()

In [ ]:
import plotly.express as px
px.scatter(df, x='vol', y='change_abs',trendline='ols')

# THE END... but not really because I'll be done when I'm rich or dead.